# Importing BigQuery
We'll use a Python package called bq_helper. It has functions for putting BigQuery results in Pandas DataFrames.

In [ ]:
import bq_helper
# create a helper object for our bigquery dataset
chicago_crime = bq_helper.BigQueryHelper(active_project= "bigquery-public-data", 
                                       dataset_name = "chicago_crime")

 An example of address of BigQuery datasets:
 BigQuery Table [bigquery-public-data](This is BigQuery info, always the same)  [hacker_news](dataset) [comments](Table)  [](https://www.kaggle.com/dansbecker/getting-started-with-sql-and-bigquery).
 We will need to pass this information to BigQueryHelper in order to create our helper object. The active_project argument takes the BigQuery info, which is currently "bigquery-public-data" for all the BigQuery datasets on Kaggle. The dataset_name argument takes the name of the dataset we've added to our query. In this case it's "hacker_news". So we can create our BigQueryHelper object like so:


## Exploring Structure of datasets

Schema: A description of how data is organized within a dataset.
We can use the BigQueryHelper.list_tables() method to list all the files in the chicago crime dataset.
BigQuery datasets can be very large, and there are some restrictions on how much data you can access.

In [4]:
chicago_crime.list_tables() #printing tables in datasets

['crime']

In [5]:
chicago_crime.table_schema("crime") #printing all info from table

[SchemaField('unique_key', 'INTEGER', 'REQUIRED', 'Unique identifier for the record.', ()),
 SchemaField('case_number', 'STRING', 'NULLABLE', 'The Chicago Police Department RD Number (Records Division Number), which is unique to the incident.', ()),
 SchemaField('date', 'TIMESTAMP', 'NULLABLE', 'Date when the incident occurred. this is sometimes a best estimate.', ()),
 SchemaField('block', 'STRING', 'NULLABLE', 'The partially redacted address where the incident occurred, placing it on the same block as the actual address.', ()),
 SchemaField('iucr', 'STRING', 'NULLABLE', 'The Illinois Unifrom Crime Reporting code. This is directly linked to the Primary Type and Description. See the list of IUCR codes at https://data.cityofchicago.org/d/c7ck-438e.', ()),
 SchemaField('primary_type', 'STRING', 'NULLABLE', 'The primary description of the IUCR code.', ()),
 SchemaField('description', 'STRING', 'NULLABLE', 'The secondary description of the IUCR code, a subcategory of the primary descriptio

Each SchemaField tells us about a specific column. In order, the information is:

* The name of the column
* The datatype in the column
* The mode of the column (NULLABLE means that a column allows NULL values, and is the default)
* A description of the data in that column


In [8]:
chicago_crime.head("crime") 

,unique_key,case_number,date,block,iucr,primary_type,description,location_description,arrest,domestic,beat,district,ward,community_area,fbi_code,x_coordinate,y_coordinate,year,updated_on,latitude,longitude,location
0,5036970,HM636584,2006-10-02 14:45:00+00:00,092XX S LONGWOOD DR,1562,SEX OFFENSE,AGG CRIMINAL SEXUAL ABUSE,STREET,True,False,2221,22,19,72,17,1165834.0,1843597.0,2006,2018-02-28 15:56:25+00:00,41.726411,-87.668129,"(41.726410782, -87.668128711)"
1,9114724,HW257681,2013-05-02 18:56:00+00:00,013XX E 79TH ST,0331,ROBBERY,ATTEMPT: AGGRAVATED,SMALL RETAIL STORE,False,False,411,4,8,45,03,1186404.0,1852879.0,2013,2018-02-10 15:50:01+00:00,41.751421,-87.592487,"(41.751420646, -87.592486791)"
2,5087600,HM690882,2006-10-30 12:30:00+00:00,080XX S KENWOOD AVE,0484,BATTERY,PRO EMP HANDS NO/MIN INJURY,"SCHOOL, PUBLIC, BUILDING",False,False,411,4,8,45,08B,1186607.0,1851902.0,2006,2018-02-28 15:56:25+00:00,41.748735,-87.591774,"(41.748734863, -87.591773761)"
3,9139778,HW284479,2013-05-18 11:00:00+00:00,035XX S LAKE PARK AVE,0496,BATTERY,AGGRAVATED DOMESTIC BATTERY: KNIFE/CUTTING INST,APARTMENT,False,True,212,2,4,36,04B,1181941.0,1881755.0,2013,2018-02-10 15:50:01+00:00,41.830763,-87.607949,"(41.830763232, -87.607948532)"
4,2911242,HJ589760,2003-08-27 04:20:00+00:00,020XX W 37TH ST,0330,ROBBERY,AGGRAVATED,SIDEWALK,False,False,913,9,11,59,03,1163546.0,1880140.0,2003,2018-02-28 15:56:25+00:00,41.826738,-87.675486,"(41.826737901, -87.675485909)"


In [9]:
chicago_crime.head("crime", selected_columns = "date", num_rows = 10) 

,date
0,2006-10-02 14:45:00+00:00
1,2013-05-02 18:56:00+00:00
2,2006-10-30 12:30:00+00:00
3,2013-05-18 11:00:00+00:00
4,2003-08-27 04:20:00+00:00
5,2003-08-25 08:00:00+00:00
6,2003-09-03 20:30:00+00:00
7,2017-07-14 22:50:00+00:00
8,2006-08-26 18:37:11+00:00
9,2006-11-27 20:30:00+00:00


What's a **query**? A query is small piece of SQL code that specifies what data would you like to scan from a databases, and how much of that data you would like returned. (Note that your quota is on data scanned, not the amount of data returned.)

One way to help avoid this is to estimate how big your query will be before you actually execute it. You can do this with the BigQueryHelper.estimate_query_size() method. 

In [12]:
query = """SELECT block FROM `bigquery-public-data.chicago_crime.crime`
WHERE primary_type = "BATTERY" """
chicago_crime.estimate_query_size(query)

0.19992566481232643

# Running query
You have two methods available to help you do this:

* -BigQueryHelper.query_to_pandas(query): This method takes a query and returns a Pandas dataframe.
* -BigQueryHelper.query_to_pandas_safe(query, max_gb_scanned=1): This method takes a query and returns a Pandas dataframe only if the size of the query is less than the upperSizeLimit **(1 gigabyte by default)**.

## Best practices can help you avoid trouble down the line:

* Avoid using the asterisk (*) in your queries. The asterisk means “everything”. This may be okay with smaller datasets, but getting everything from a 4 terabyte dataset takes a long time and eats into your monthly usage limit.
* For initial exploration, look at just part of the table instead of the whole thing. If you're just curious to see what data's in a table, preview it instead of scanning the whole table. The BigQueryHelper.head() method in our helper package does this. Like head() in Pandas or R, it returns just the first few rows for you to look at.
* Double-check the size of complex queries. If you're planning on running what might be a large query, either estimate the size first or run it using the BigQueryHelper.query_to_pandas_safe() method.
* Be cautious about joining tables. In particular, avoid joining a table with itself (i.e. a self-join) and try to avoid joins that return a table that's larger than the ones you're joining together. (You can double-check yourself by joining just the heads of the tables.)
* Don't rely on LIMIT: One of the things that can be confusing when working with BigQuery datasets is the difference between the data you scan and the data you actually get back especially since it's the first one that actually counts against your quota. When you do something like select a column with LIMIT = 10, you'll only get 10 results back, but you'll scan the whole column (and that counts against your monthly usage limit).